# Taller Práctico 4: Construcción de Modelos Analíticos de Pronóstico de Ventas


**Objetivos del Taller:**

* Construir un modelo de regresión lineal para predecir el valor total de las ventas futuras.
* Realizar un pronóstico de ventas diferenciando el canal de ventas.
* Utilizar métricas estadísticas para evaluar la precisión y la utilidad del modelo de pronóstico de ventas.

# Introducción al Pronóstico de Ventas

El pronóstico de ventas es una técnica analítica esencial para prever el volumen de ventas futuro basado en datos históricos y tendencias de mercado. Este proceso es vital para la planificación estratégica en negocios, permitiendo a las empresas anticipar ingresos, gestionar inventarios, optimizar recursos y planificar futuras campañas de marketing. A continuación, se detallan algunos beneficios clave del pronóstico de ventas:

* **Planificación Financiera y de Inventario:** El pronóstico de ventas ayuda a las empresas a prever los ingresos futuros y a gestionar mejor el inventario, reduciendo el exceso de stock y evitando rupturas de stock.

* **Toma de Decisiones Estratégicas:**  Con un pronóstico confiable, las empresas pueden tomar decisiones más informadas sobre expansión, inversiones y estrategias de mercado, adaptándose mejor a las dinámicas del mercado.

* **Eficiencia Operativa:** Al prever la demanda futura, las empresas pueden optimizar la asignación de recursos a la producción y la logística, mejorando la eficiencia operativa.

# Construcción de un Modelo Analítico Orientado al Pronóstico de Ventas

## Instalar las bibliotecas necesarias

Lo primero que debemos hacer es asegurarnos de tener todas las bibliotecas necesarias instaladas y cargadas en nuestro entorno de trabajo. Estas bibliotecas proporcionan las herramientas fundamentales para manipular datos, realizar análisis estadísticos, construir y evaluar modelos de machine learning. A continuación, se muestran las bibliotecas que utilizaremos a lo largo de este notebook:

* **numpy:** Biblioteca fundamental para operaciones matemáticas avanzadas y manipulación de arreglos en Python.

* **pandas:** Biblioteca poderosa para el análisis de datos, especialmente útil para trabajar con datos en formato de tablas (DataFrames).

* **seaborn:** Biblioteca para visualización de datos que facilita la creación de gráficos estadísticos claros y atractivos.

* **matplotlib:** Biblioteca esencial para crear visualizaciones y gráficos personalizados en Python.

* **sklearn:** Conjunto de herramientas de machine learning en Python que incluye algoritmos para clasificación, regresión y agrupamiento, entre otros.

In [ ]:
#!pip install numpy pandas seaborn matplotlib scikit-learn

## Cargar las bibliotecas previamente instaladas

Ahora que tenemos instaladas todas las bibliotecas necesarias, procedemos a cargarlas en nuestro entorno de trabajo.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from datetime import datetime
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

## Cargar el conjunto de datos

Con las bibliotecas listas, el siguiente paso es cargar nuestro conjunto de datos que utilizaremos para la construcción de nuestro modelo de pronóstico de ventas.

In [ ]:
# Cargar los datos desde un archivo CSV
data = pd.read_csv('./data/customer_transactions_dataset.csv')

# Realizar una copia de los datos para la etapa de entendimiento
data_exploration = data.copy()

# Visualizar las primeras filas del dataframe
data.head()

## Entendimiento del conjunto de datos

Una vez cargado el conjunto de datos, es fundamental tener un entendimiento las variables clave que utilizaremos para construir nuestro modelo de pronóstico de ventas. Aunque en talleres anteriores ya hemos explorado en profundidad la estructura y distribución de los datos, en este caso nos enfocaremos únicamente en las variables de interés para este análisis.

En particular, estudiaremos:

* `Valor`: El monto de las ventas realizadas.
* `Fecha de Compra`: La fecha en que se registraron las transacciones.
* `Canal de Venta`: El medio a través del cual se efectuaron las ventas.

Comenzaremos analizando la evolución del valor de las ventas a lo largo del tiempo.

### Convertir la columna Fecha de Compra a Date

Antes de comenzar con la exploración inicial de la evolución de las ventas, es importante asegurarnos de que la columna Fecha de Compra esté en un formato de fecha adecuado. En Pandas, esto implica convertir esta columna al tipo de dato datetime, lo que facilitará operaciones como agrupaciones, filtrado por rangos de fechas y la generación de visualizaciones temporales. 

In [ ]:
# Convertir la columna 'Fecha de Compra' a formato de fecha
data_exploration['Fecha de Compra'] = pd.to_datetime(data_exploration['Fecha de Compra'], errors='coerce')
data.head()

### Agrupar Ventas por Fecha

El siguiente paso para analizar la evolución de las ventas a lo largo del tiempo es definir la granularidad que utilizaremos en nuestro modelo. En este caso, trabajaremos con una granularidad diaria, lo que nos permitirá observar el comportamiento de las ventas día a día.

Para ello, agruparemos los datos por la columna Fecha de Compra y calcularemos el valor total de ventas para cada día. Este enfoque nos proporcionará una visión clara de cómo varían las ventas a nivel diario, sentando las bases para un análisis más detallado.

In [ ]:
# Agrupar datos por fecha para obtener el valor total de transacciones
data_by_date = data_exploration.groupby('Fecha de Compra').agg({'Valor': 'sum'}).reset_index()
data_by_date.head()

### Evolución de las ventas

Una vez que hemos agrupado las ventas por fecha, el siguiente paso es visualizar la evolución del valor total de las transacciones a lo largo del año 2024. Utilizaremos un gráfico de línea para representar cómo varían las ventas a lo largo del tiempo.

In [ ]:
# Gráfico de línea para visualizar la evolución del valor total de transacciones
plt.figure(figsize=(12, 6))
plt.plot(data_by_date['Fecha de Compra'], data_by_date['Valor'], marker='o')
plt.title('Evolución del Valor de Transacciones a lo Largo del Tiempo')
plt.xlabel('Fecha de Compra')
plt.ylabel('Valor Total de Transacciones')
plt.xticks(rotation=45)
plt.grid()
plt.show()

En el gráfico anterior, podemos observar que no existe una tendencia clara en la evolución de las ventas. Sin embargo, es evidente que los datos correspondientes al mes de noviembre están incompletos. Este detalle es crucial, ya que incluir solo una fracción de los datos de noviembre podría sesgar el análisis, especialmente si más adelante se decide trabajar con una granularidad mensual.

Por este motivo, sería recomendable limitar el conjunto de datos hasta el mes de octubre durante la etapa de preparación, para evitar que el modelo interprete erróneamente una caída en las ventas totales.

A continuación, se presenta una versión actualizada del gráfico, resaltando en rojo el mes de noviembre para evidenciar que solo se cuenta con una fracción de los datos de este mes.

In [ ]:
# Gráfico de línea para visualizar la evolución del valor total de transacciones
plt.figure(figsize=(12, 6))
plt.plot(data_by_date['Fecha de Compra'], data_by_date['Valor'], marker='o')
plt.axvspan(pd.Timestamp('2024-11-01'), pd.Timestamp(f'2024-11-30'), color='red', alpha=0.3, label='Noviembre')
plt.title('Evolución del Valor de Transacciones a lo Largo del Tiempo')
plt.xlabel('Fecha de Compra')
plt.ylabel('Valor Total de Transacciones')
plt.xticks(rotation=45)
plt.grid()
plt.legend()
plt.show()

### Evolución de Ventas por Canal

Dado que no se observó una tendencia clara en la evolución general de las ventas, analizaremos si diferenciarlas por canal de venta permite identificar patrones más definidos. Esto tiene sentido, ya que se espera que cada canal tenga un comportamiento distinto y características particulares.

Para ello, agruparemos los datos tanto por fecha de compra como por canal de venta, calculando el valor total de las ventas para cada combinación.

In [ ]:
# Agrupar datos por fecha para obtener el valor total de transacciones.
data_by_date_channel = data_exploration.groupby(['Fecha de Compra', 'Canal de Venta']).agg({'Valor': 'sum'}).reset_index()
data_by_date_channel.head()

A continuación, se genera un gráfico de líneas similar al realizado previamente, pero en esta ocasión se diferencian las ventas por canal de venta utilizando la agrupación previamente calculada. Este gráfico nos permite visualizar cómo varía el valor total de las transacciones a lo largo del tiempo para cada canal de venta.

In [ ]:
# Crear un gráfico de línea con seaborn
plt.figure(figsize=(12, 6))
sns.lineplot(data=data_by_date_channel, x='Fecha de Compra', y='Valor', hue='Canal de Venta')

plt.title('Evolución del Valor de Transacciones por Canal a lo Largo del Tiempo')
plt.xlabel('Fecha de Compra')
plt.ylabel('Valor Total de Transacciones')
plt.xticks(rotation=45)
plt.grid(True)
plt.legend(title='Canal de Venta')
plt.show()

## Preparación de los Datos

Es crucial preparar los datos para garantizar su calidad y utilidad. En esta etapa, llevamos a cabo un proceso de limpieza y transformación para asegurar que la información sea consistente y esté alineada con los objetivos del proyecto. Esto incluye la eliminación de registros duplicados y valores nulos, la conversión de fechas al formato adecuado, y el filtrado de datos incompletos que podrían distorsionar los resultados. Además, reorganizamos las ventas para facilitar su análisis, creando nuevas variables clave que permitirán una exploración más profunda en las siguientes etapas. Estos pasos nos proporcionan un conjunto de datos limpio, estructurado y listo para el modelado.

### Eliminación de registros duplicados

El primer paso en la preparación de datos es identificar y eliminar registros duplicados. Los duplicados pueden surgir debido a errores en la recolección o procesamiento de los datos, y su presencia podría distorsionar los análisis al sobrerrepresentar ciertos valores. Este paso garantiza que cada fila del conjunto de datos represente una observación única.

In [ ]:
# Eliminar registros duplicados en el conjunto de datos
data = data.drop_duplicates()
data.head()

### Eliminación de registros con valores nulos

A continuación, eliminamos las filas con valores nulos. Esto asegura que las métricas calculadas no se vean afectadas por datos incompletos. Es importante evaluar previamente si la eliminación de estas filas afecta significativamente el tamaño de la muestra.

In [ ]:
# Eliminar filas con valores nulos en el conjunto de datos
data = data.dropna()
data.head()

### Conversión de Fechas al Formato Adecuado

Para facilitar las operaciones temporales, convertimos la columna `Fecha de Compra` al formato de fecha reconocido por Pandas. Esto permite realizar cálculos y manipulaciones con mayor precisión, como agrupaciones por rangos de fechas o la creación de nuevas variables temporales. Durante este proceso, manejamos errores que podrían surgir debido a valores mal formateados.

In [ ]:
data['Fecha de Compra'] = pd.to_datetime(data['Fecha de Compra'], errors='coerce')
data.head()

### Filtrado de Datos de Noviembre

Como se observó previamente, los datos correspondientes al mes de noviembre están incompletos. Para evitar sesgos en el análisis y asegurar la calidad de los resultados, filtramos el conjunto de datos para incluir únicamente las transacciones realizadas hasta octubre. Este paso es especialmente relevante si en el futuro se decide trabajar con granularidades mensuales.

In [ ]:
data = data[data["Fecha de Compra"]<"2024-11-01"]
data.head()

### Agrupación de Ventas por Fecha y Canal

Una vez limpios, los datos se agrupan por fecha de compra y canal de venta. Este proceso consolida las ventas realizadas en un mismo día y canal, calculando el valor total de las transacciones. Esta reorganización nos permite observar patrones específicos y comportamientos únicos para cada canal de venta.

In [ ]:
data = data.groupby(['Fecha de Compra', 'Canal de Venta']).agg({'Valor': 'sum'}).reset_index()
data.head()

### Creación de una Variable Temporal

Finalmente, añadimos una nueva columna que representa el tiempo en días desde la primera fecha registrada en el conjunto de datos. Esta variable es esencial para modelar las ventas como una serie temporal, ya que transforma las fechas en un formato numérico que los modelos estadísticos y de aprendizaje automático pueden procesar fácilmente.

In [ ]:
# Preparar los datos para la regresión lineal
data['Tiempo'] = (data['Fecha de Compra'] - data['Fecha de Compra'].min()).dt.days
data.head()

## Construcción del Modelo de Pronóstico de Ventas utilizando Regresión Lineal

En esta sección, abordaremos la construcción de un modelo de regresión lineal para pronosticar las ventas. La regresión lineal es un modelo estadístico utilizado para identificar la relación entre una variable dependiente (en este caso, el valor de las ventas) y una o más variables independientes (como el tiempo). Este modelo busca ajustar una línea recta que minimice la suma de los errores al cuadrado entre las predicciones y los valores reales.

El modelo sigue la fórmula general:

$$
y=\beta_0+\beta_1x+\epsilon
$$

Donde:

* $y$ es la variable dependiente (valor de las ventas).
* $\beta_0$ es el intercepto (el valor de $y$ cuando $x=0$).
* $\beta_1$ es la pendiente de la línea, que mide cómo cambia $y$ por cada unidad de cambio en $x$.
* $x$ es la variable independiente (en este caso, el tiempo).
* $\epsilon$ es el término de error, que captura las desviaciones no explicadas por el modelo.

El objetivo del modelo es encontrar los valores óptimos de $\beta_0$ y $\beta_1$ que minimicen la diferencia entre los valores observados y los valores predichos.

### Proceso de Construcción del Modelo

A continuación, se describen los pasos a seguir para construir el modelo de pronóstico de ventas utilizando regresión lineal.

#### 1. Filtrado por Canal de Venta

Como observamos en la etapa de entendimiento, cada canal puede tener un comportamiento diferente, por esta razón, comenzamos seleccionando los datos correspondientes a un canal específico. Esto permite crear un modelo personalizado para cada segmento, aumentando la precisión del pronóstico.

#### 2. División de Datos en Entrenamiento y Prueba

Para evaluar el desempeño del modelo, dividimos los datos en dos subconjuntos:

* **Datos de entrenamiento:** Incluyen las ventas realizadas antes de julio de 2024. Estos datos se utilizan para ajustar el modelo.
* **Datos de prueba:** Incluyen las ventas posteriores a julio de 2024. Este conjunto sirve para medir la capacidad del modelo de generalizar sus predicciones a datos nuevos.


#### 3. Selección de Variables

En este modelo, usamos como variable independiente el tiempo, que representa los días desde la primera fecha registrada. La variable dependiente es el monto total de las ventas.

#### 4. Entrenamiento del Modelo

Utilizamos el conjunto de datos de entrenamiento para ajustar un modelo de regresión lineal. Durante este paso, el modelo aprende los valores óptimos de los parámetros que mejor explican la relación entre el tiempo y el valor de las ventas.

#### 5. Generación de Predicciones

Con el modelo entrenado, generamos predicciones para ambos conjuntos de datos (entrenamiento y prueba). Esto nos permite evaluar qué tan bien se ajusta el modelo a los datos de entrenamiento y qué tan precisas son sus predicciones para datos no vistos anteriormente.

In [ ]:
def train_test_linear_regression(data, channel):
    """
    Realiza el entrenamiento y evaluación de un modelo de regresión lineal
    basado en datos filtrados por un canal de venta específico.
    
    Args:
        data (pd.DataFrame): DataFrame con los datos de entrada. Debe contener
                             las columnas 'Canal de Venta', 'Fecha de Compra', 
                             'Tiempo' y 'Valor'.
        channel (str): Canal de venta por el cual se filtrarán los datos.
        
    Returns:
        tuple: Contiene los siguientes elementos:
            - train (pd.DataFrame): Datos de entrenamiento con predicciones.
            - test (pd.DataFrame): Datos de prueba con predicciones.
            - model (LinearRegression): Modelo de regresión lineal entrenado.
    """
    # Filtrar los datos por el canal de venta especificado
    channel_data = data[data['Canal de Venta'] == channel]
    
    # Dividir los datos en conjuntos de entrenamiento y prueba según la fecha
    train = channel_data[channel_data['Fecha de Compra'] < datetime(2024, 7, 1)].copy()
    test = channel_data[channel_data['Fecha de Compra'] >= datetime(2024, 7, 1)].copy()
    
    # Separar las variables independientes (X) y dependientes (y) para entrenamiento y prueba
    X_train = train[['Tiempo']]
    y_train = train['Valor']
    X_test = test[['Tiempo']]
    y_test = test['Valor']
    
    # Crear y entrenar un modelo de regresión lineal
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Generar predicciones para los datos de entrenamiento y prueba
    train.loc[:, 'Predicción'] = model.predict(X_train)
    test.loc[:, 'Predicción'] = model.predict(X_test)
    
    return channel, train, test, model

### Visualización de las Predicciones por Canal de Venta

#### Predicción para un Canal Específico

Para evaluar el desempeño del modelo de regresión lineal en un canal específico, primero aplicamos la función de entrenamiento y prueba al canal "Tienda Física". Esto nos permite analizar cómo las ventas históricas se ajustan al modelo y cómo se comportan las predicciones en comparación con los datos reales.

In [ ]:
channel, train_tienda, test_tienda, model_tienda = train_test_linear_regression(data, "Tienda Física")

Después, generamos un gráfico para visualizar la evolución de las ventas y las predicciones. En el gráfico se diferencian las siguientes curvas:

* **Ventas de entrenamiento:** Representan los datos históricos utilizados para ajustar el modelo.
* **Ventas de prueba:** Muestran los datos no vistos que se utilizan para evaluar el modelo.
* **Predicciones del modelo:** Se visualizan para los conjuntos de entrenamiento y prueba, permitiendo analizar qué tan bien el modelo se ajusta a los datos reales.

Este gráfico nos proporciona una visión inicial de la capacidad del modelo para capturar la tendencia de las ventas y su comportamiento frente a datos no utilizados durante el entrenamiento.

In [ ]:
plt.figure(figsize=(14, 7))
plt.plot(train_tienda['Fecha de Compra'], train_tienda['Valor'], label='Ventas (Entrenamiento)', alpha=0.8)
plt.plot(test_tienda['Fecha de Compra'], test_tienda['Valor'], label='Ventas (Prueba)', alpha=0.8)
plt.plot(train_tienda['Fecha de Compra'], train_tienda['Predicción'], color='green', linestyle='--', linewidth=2, label='Predicción (Entrenamiento)')
plt.plot(test_tienda['Fecha de Compra'], test_tienda['Predicción'], color='red', linestyle='--', linewidth=2, label='Predicción (Prueba)')
plt.title(f'Evolución del Valor de Transacciones con Predicción ({channel})')
plt.xlabel('Fecha de Compra')
plt.ylabel('Valor Total de Transacciones')
plt.xticks(rotation=45)
plt.grid()
plt.legend()
plt.show()

#### Predicción para Todos los Canales

A continuación, aplicamos el mismo enfoque a todos los canales de venta presentes en el conjunto de datos. Esto incluye filtrar los datos por cada canal, entrenar un modelo independiente y generar las predicciones correspondientes.

Para visualizar los resultados de manera eficiente, creamos gráficos en un formato de subplots de 2x2, donde cada gráfico representa la evolución de las ventas y las predicciones para un canal. Cada subplot muestra:

* **Las ventas reales de entrenamiento y prueba.**
* **Las predicciones del modelo para ambos conjuntos de datos.**

Además, formateamos los ejes de tiempo para resaltar los meses, facilitando la interpretación de las tendencias temporales en cada canal.

In [ ]:
# Datos para los 4 canales
channels = data["Canal de Venta"].unique()
data_train_test = [
    train_test_linear_regression(data, channel) for channel in channels
]

# Crear subplots 2x2
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for i, (channel, (channel, train_data, test_data, model)) in enumerate(zip(channels, data_train_test)):
    ax = axes[i]
    # Graficar datos de entrenamiento, prueba y predicciones
    ax.plot(train_data['Fecha de Compra'], train_data['Valor'], label='Ventas (Entrenamiento)', alpha=0.8)
    ax.plot(test_data['Fecha de Compra'], test_data['Valor'], label='Ventas (Prueba)', alpha=0.8)
    ax.plot(train_data['Fecha de Compra'], train_data['Predicción'], color='green', linestyle='--', linewidth=2, label='Predicción (Entrenamiento)')
    ax.plot(test_data['Fecha de Compra'], test_data['Predicción'], color='red', linestyle='--', linewidth=2, label='Predicción (Prueba)')
    
    # Títulos y configuración
    ax.set_title(f'Evolución de Ventas: {channel}')
    ax.set_xlabel('Fecha de Compra')
    ax.set_ylabel('Valor Total de Transacciones')
    ax.grid()
    
    # Formatear eje X para mostrar solo el mes
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
    ax.xaxis.set_major_locator(mdates.MonthLocator())
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)

# Leyenda única
handles, labels = axes[0].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center', ncol=4, bbox_to_anchor=(0.5, 0.98))

# Ajustar espacio entre subplots
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

## Evaluación Cuantitativa de los Modelos de Pronóstico de Ventas

Una vez entrenados los modelos y generadas las predicciones para cada canal de venta, es fundamental evaluar su desempeño de manera cuantitativa. En esta sección, utilizamos métricas estándar para medir la precisión y calidad de los pronósticos. Las métricas consideradas son:

* **Root Mean Squared Error (RMSE)**: Es la raíz cuadrada del promedio de los errores al cuadrado, otorgando un mayor peso a los errores más grandes. Esta métrica es adecuada para identificar si hay grandes desviaciones en las predicciones.

* **Mean Absolute Error (MAE)**: Representa el error promedio absoluto entre las predicciones y los valores reales. Es útil para comprender, en términos absolutos, cuánto se desvían las predicciones del modelo.

* **Mean Absolute Percentage Error (MAPE)**: Calcula el porcentaje promedio de error relativo entre las predicciones y los valores reales, permitiendo comparar el desempeño del modelo entre diferentes canales de venta, independientemente de las magnitudes.

A continuación, para cada canal de venta, calculamos las métricas mencionadas utilizando los valores reales y las predicciones en el conjunto de prueba. Los resultados se almacenan en un DataFrame, lo que facilita su visualización y análisis comparativo entre los canales.

In [ ]:
# Diccionario para almacenar métricas por canal
metrics = {
    "Canal": [],
    "MAE": [],
    "RMSE": [],
    "MAPE": []
}

# Calcular métricas para cada canal
for channel, (channel, train_data, test_data, model) in zip(channels, data_train_test):
    mae = mean_absolute_error(test_data['Valor'], test_data['Predicción'])
    rmse = np.sqrt(mean_squared_error(test_data['Valor'], test_data['Predicción']))
    mape = np.mean(np.abs((test_data['Valor'] - test_data['Predicción']) / test_data['Valor'])) * 100
    
    metrics["Canal"].append(channel)
    metrics["MAE"].append(mae)
    metrics["RMSE"].append(rmse)
    metrics["MAPE"].append(mape)


# Convertir métricas a DataFrame para facilitar la visualización
metrics_df = pd.DataFrame(metrics)
metrics_df

### Visualización de las Métricas

Para interpretar mejor los resultados, a continuación graficamos las métricas MAE y MAPE para cada canal de venta en un formato de barras:

* **MAE por canal:** Este gráfico muestra el error absoluto promedio para cada canal, destacando los canales donde el modelo tiene mayor precisión.

* **MAPE por canal:** Este gráfico permite evaluar el error relativo porcentual, facilitando comparaciones entre canales con diferentes volúmenes de ventas.

In [ ]:
# Creando un único subplot con MAE y MAPE
fig, ax = plt.subplots(1, 2, figsize=(14, 6))

# Gráfica de MAE
ax[0].bar(metrics_df["Canal"], metrics_df["MAE"], color='skyblue', alpha=0.7)
ax[0].set_title("MAE por Canal")
ax[0].set_xlabel("Canal")
ax[0].set_ylabel("MAE")
ax[0].tick_params(axis='x', rotation=45)
ax[0].grid(axis="y", linestyle="--", alpha=0.7)

# Gráfica de MAPE
ax[1].bar(metrics_df["Canal"], metrics_df["MAPE"], color='orange', alpha=0.7)
ax[1].set_title("MAPE por Canal")
ax[1].set_xlabel("Canal")
ax[1].set_ylabel("MAPE")
ax[1].tick_params(axis='x', rotation=45)
ax[1].grid(axis="y", linestyle="--", alpha=0.7)

# Ajustando el diseño
plt.tight_layout()
plt.show()

# Análisis Cualitativo de los Resultados

### Canal: Distribuidor

El modelo presenta un MAE de 28,365 y un MAPE del 23.14\%, lo que significa que, en promedio, las predicciones diarias tienen un error absoluto de $28,365, representando aproximadamente un 23\% de los valores reales. Esto sugiere que las ventas en este canal son altamente variables y difíciles de predecir. Este comportamiento es típico de canales donde las transacciones tienden a ser menos frecuentes pero de mayor volumen, como ocurre con los distribuidores.

### Canal: En Línea

Para el canal en línea, el modelo tiene un MAE de 30,582 y un MAPE de 13.69\%. Aunque el error absoluto es el más alto entre todos los canales, el error relativo es moderado, lo que indica que las predicciones son razonablemente precisas en proporción al alto volumen general de ventas en este canal. Esto sugiere que las ventas en línea son algo más regulares, aunque las fluctuaciones significativas aún representan un desafío para el modelo.

### Canal: Telefónica
En este canal, el modelo logra un MAE de 23,107 y un MAPE de 16.38\%. Estos valores reflejan un desempeño intermedio, con errores absolutos y relativos moderados. Esto sugiere que las ventas telefónicas son más predecibles que las del canal Distribuidor, pero menos regulares que las de Tienda Física. La variabilidad en este canal podría estar relacionada con la dependencia de la interacción humana para concretar ventas.

### Canal: Tienda Física
El canal Tienda Física tiene el mejor desempeño con un MAE de 24,365 y un MAPE de 9.14\%. Esto indica que las predicciones tienen un error absoluto promedio aceptable y un error relativo significativamente bajo. Este resultado refleja la consistencia y regularidad en las ventas de este canal, donde los patrones parecen ser más estables y, por lo tanto, más fáciles de modelar.